In [56]:
%reset -f
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
from linearmodels.iv import IV2SLS
import functions as fun
import statsmodels.formula.api as sm
import seaborn as sns
from matplotlib import pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [57]:
df = pd.read_csv('dataset.csv')
Nobs=df['ID'].count()
df['Intercept']=np.ones((Nobs,1))
df.rename(columns={'Market share':'Market_share'}, inplace=True)
df2 = df[df['Market_share'] != 0]

df2.head(20)

,ID,Year,Market_share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,Segment,Country,Sales,Intercept
8,1,2021,0.010373,Aiways,U5,400,284621.700,201,34,SUV,C,CN,257,1.0
9,1,2022,0.005976,Aiways,U5,400,313681.829,201,34,SUV,C,CN,183,1.0
10,1,2023,0.002860,Aiways,U5,400,264524.000,201,34,SUV,C,CN,177,1.0
21,2,2023,0.000048,Aiways,U6,405,360638.000,214,34,SUV,C,CN,3,1.0
28,3,2019,0.040630,Audi,e-tron,375,979704.475,402,17,SUV,F,DE,222,1.0
29,3,2020,0.034680,Audi,e-tron,375,890101.410,402,17,SUV,F,DE,491,1.0
30,3,2021,0.010494,Audi,e-tron,375,800035.193,402,17,SUV,F,DE,260,1.0
31,3,2022,0.017570,Audi,e-tron,375,789723.656,402,17,SUV,F,DE,538,1.0
32,3,2023,0.001099,Audi,e-tron,375,673037.728,402,17,SUV,F,DE,68,1.0
41,4,2021,0.003391,Audi,e-tron GT,472,1278896.110,522,17,Sedan,F,DE,84,1.0


In [58]:
# Copy the dataframe
data = df2.copy().reset_index(drop=True)
data.head(20)

,ID,Year,Market_share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,Segment,Country,Sales,Intercept
0,1,2021,0.010373,Aiways,U5,400,284621.700,201,34,SUV,C,CN,257,1.0
1,1,2022,0.005976,Aiways,U5,400,313681.829,201,34,SUV,C,CN,183,1.0
2,1,2023,0.002860,Aiways,U5,400,264524.000,201,34,SUV,C,CN,177,1.0
3,2,2023,0.000048,Aiways,U6,405,360638.000,214,34,SUV,C,CN,3,1.0
4,3,2019,0.040630,Audi,e-tron,375,979704.475,402,17,SUV,F,DE,222,1.0
5,3,2020,0.034680,Audi,e-tron,375,890101.410,402,17,SUV,F,DE,491,1.0
6,3,2021,0.010494,Audi,e-tron,375,800035.193,402,17,SUV,F,DE,260,1.0
7,3,2022,0.017570,Audi,e-tron,375,789723.656,402,17,SUV,F,DE,538,1.0
8,3,2023,0.001099,Audi,e-tron,375,673037.728,402,17,SUV,F,DE,68,1.0
9,4,2021,0.003391,Audi,e-tron GT,472,1278896.110,522,17,Sedan,F,DE,84,1.0


In [59]:
#Scale for better intepretation

data['Price'] = data['Price']/10_000 #(Change in ms(%) for change in pris in 10.000)
data['HP'] = data['HP']/10           #(Change in ms(%) for change in HP in 10)
data['Range'] = data['Range']/10     #(Change in ms(%) for change in rækkevidde in 10)

In [60]:
# Creating dummy for china
data['China'] = (data['Country'] == 'CN').astype(int)

# OLS

In [61]:
OLS = sm.logit('Market_share ~ 1 + Price + Range + HP + Chargetime + China', data).fit(cov_type='HC3')
OLS.summary()#.tables[1]

Optimization terminated successfully.
         Current function value: 0.056167
         Iterations 10


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:           Market_share   No. Observations:                  334
Model:                          Logit   Df Residuals:                      328
Method:                           MLE   Df Model:                            5
Date:                Sun, 12 May 2024   Pseudo R-squ.:                  -1.598
Time:                        13:59:43   Log-Likelihood:                -18.760
converged:                       True   LL-Null:                       -7.2204
Covariance Type:                  HC3   LLR p-value:                     1.000
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -5.0717      0.671     -7.564      0.000      -6.386      -3.757
Price         -0.0498      0.008     -5.924      0.000      -0.066      -0.033
Range          0.0401      0.015      2.751      0.006       0.012       0.069
HP             0.0521      0.017      2.984      0.003       0.018       0.086
Chargetime     0.0280      0.013      2.195      0.028       0.003       0.053
China         -3.2943      0.395     -8.336      0.000      -4.069      -2.520
==============================================================================
"""

# Willingness to pay

In [62]:
alpha = OLS.params[1] # Price coefficient
beta = list(OLS.params[:1]) + list(OLS.params[2:]) # Other coefficients
beta_alpha_ratio = [b / -alpha for b in beta[1:]] #Willingness to pay (excluding constant)

for i in range(len(OLS.params.index[2:])): #[2:] to exclude constant and price
    print('W2P:', OLS.params.index[2:][i], beta_alpha_ratio[i])

W2P: Range 0.8053276159773862
W2P: HP 1.0456928541285768
W2P: Chargetime 0.5608479831897047
W2P: China -66.10144787051956


NOTE: English delimiter\
8,054 DKK for every 10 increase in Range\
10,345 DKK for every 10 increase in HP\
5,608 DKK for every 10 increase in Chargetime\
-661,014 DKK for Chinese cars OR -69,000 DKK for Chinese cars???

# Logit

In [63]:
# IMPORTANT: The data must be sorted by year and ID before running the function
#             Because the CCPs returned by the function are sorted by year and ID
logit_data = data.sort_values(['Year', 'ID']).reset_index(drop=True)
X = logit_data[['Intercept', 'Range', 'HP', 'Chargetime', 'China']]
p_j = logit_data['Price']
logit_data['CCP'] = fun.ccp(alpha, beta, data, X)

In [64]:
probability_ratio = fun.probability_ratio(logit_data, 2023)
probability_ratio

Model,U5,U6,e-tron,e-tron GT,Q4 e-tron,Q8 e-tron,I3,i4,i5,I7,...,ID.3,ID.4,ID.5,ID.7,ID.Buzz,up!,C40,EX30,XC40,Free
Model,,,,,,,,,,,,,,,,,,,,,
U5,1.0,1.478733,0.177219,0.977807,0.140146,0.198496,0.136991,0.07166,0.116136,15.666242,...,0.05281,0.041701,0.03836,0.041198,0.144477,0.053441,0.027026,0.038395,0.029247,0.284957
U6,0.676255,1.0,0.119845,0.661247,0.094774,0.134234,0.092641,0.048461,0.078537,10.594371,...,0.035713,0.0282,0.025941,0.02786,0.097704,0.03614,0.018276,0.025964,0.019778,0.192703
e-tron,5.642744,8.34411,1.0,5.517514,0.790808,1.120064,0.773007,0.404361,0.655324,88.400596,...,0.297996,0.235307,0.216458,0.23247,0.815249,0.301555,0.152499,0.21665,0.165034,1.607938
e-tron GT,1.022697,1.512295,0.181241,1.0,0.143327,0.203002,0.140101,0.073287,0.118772,16.021816,...,0.054009,0.042647,0.039231,0.042133,0.147757,0.054654,0.027639,0.039266,0.029911,0.291424
Q4 e-tron,7.135415,10.551372,1.264529,6.977058,1.0,1.416354,0.97749,0.511327,0.828677,111.785143,...,0.376825,0.297552,0.273717,0.293966,1.030906,0.381325,0.19284,0.273961,0.20869,2.033284
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
up!,18.712184,27.670318,3.31615,18.296902,2.622438,3.7143,2.563406,1.340923,2.173154,293.149603,...,0.988199,0.780313,0.717807,0.770907,2.703488,1.0,0.505711,0.718445,0.547277,5.332162
C40,37.001749,54.715697,6.557403,36.180566,5.185648,7.344711,5.068917,2.651561,4.297226,579.678363,...,1.95408,1.543002,1.419402,1.524403,5.345917,1.977415,1.0,1.420665,1.082193,10.543897
EX30,26.045378,38.514152,4.615729,25.46735,3.650156,5.169912,3.56799,1.866423,3.0248,408.033201,...,1.375469,1.086113,0.999111,1.073021,3.762969,1.391894,0.703896,1.0,0.761751,7.421805


In [65]:
logit_data['Model_year'] = logit_data['Model'] + '_' + logit_data['Year'].astype(str)
marginal_effects = fun.marginal_effects(logit_data, OLS)
marginal_effects

,Intercept,Price,Range,HP,Chargetime,China
Model_year,,,,,,
I3_2013,-0.271935,-0.002672,0.002152,0.002794,0.001499,-0.176634
Leaf_2013,-0.518009,-0.00509,0.004099,0.005323,0.002855,-0.33647
Zoe_2013,-0.997168,-0.009799,0.007891,0.010246,0.005496,-0.647705
Fortwo_2013,-0.341095,-0.003352,0.002699,0.003505,0.00188,-0.221557
Model S_2013,-1.196332,-0.011756,0.009467,0.012293,0.006593,-0.777072
...,...,...,...,...,...,...
up!_2023,-0.069542,-0.000683,0.00055,0.000715,0.000383,-0.045171
C40_2023,-0.135619,-0.001333,0.001073,0.001394,0.000747,-0.08809
EX30_2023,-0.096261,-0.000946,0.000762,0.000989,0.000531,-0.062526


In [66]:
marginal_effects[OLS.params.index].mean()

Intercept    -0.145404
Price        -0.001429
Range         0.001151
HP            0.001494
Chargetime    0.000801
China        -0.094446
dtype: float64

In [67]:
elasticity = fun.elasticity(logit_data, OLS)
elasticity

,Intercept,Price,Range,HP,Chargetime,China
Model_year,,,,,,
I3_2013,-4.783349,-1.175088,1.11667,0.820826,0.474513,-0.0
Leaf_2013,-4.486043,-1.118318,1.164416,0.677615,1.063103,-0.0
Zoe_2013,-3.707658,-0.630842,1.070936,0.510513,1.144277,-0.0
Fortwo_2013,-4.703913,-0.972796,0.472753,0.38668,1.555443,-0.0
Model S_2013,-3.138386,-2.219083,1.512497,2.17677,0.518885,-0.0
...,...,...,...,...,...,...
up!_2023,-5.001153,-0.859293,1.013167,0.416253,1.322985,-0.0
C40_2023,-4.932223,-2.087279,1.818861,2.037376,0.761104,-0.0
EX30_2023,-4.973515,-1.799694,1.869511,1.369622,0.767476,-0.0


In [68]:
elasticity[OLS.params.index].mean()

Intercept    -4.904645
Price        -2.270546
Range         1.530423
HP            1.267938
Chargetime    0.918897
China        -0.384241
dtype: float64

# Cost

In [69]:
logit_data['Cost']=fun.cost_original(logit_data, alpha)
logit_data

,ID,Year,Market_share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,Segment,Country,Sales,Intercept,China,CCP,Model_year,Cost
0,10,2013,0.002237,BMW,I3,29.5,25.00000,16.7,18,Hatchback,B,DE,1,1.0,0,0.056850,I3_2013,24.123369
1,132,2013,0.472036,Nissan,Leaf,32.8,25.36900,14.7,43,Hatchback,C,JP,211,1.0,0,0.115471,Leaf_2013,24.937405
2,158,2013,0.205817,Renault,Zoe,36.5,17.31500,13.4,56,Hatchback,B,FR,92,1.0,0,0.268948,Zoe_2013,17.129697
3,167,2013,0.002237,Smart,Fortwo,12.7,21.04575,8.0,60,Hatchback,A,DE,1,1.0,0,0.072513,Fortwo_2013,20.358470
4,173,2013,0.250559,Tesla,Model S,60.9,71.95631,67.5,30,Liftback,F,US,112,1.0,0,0.381193,Model S_2013,71.825571
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329,187,2023,0.001277,Volkswagen,up!,25.6,17.48530,8.1,48,Hatchback,A,DE,79,1.0,0,0.013905,up!_2023,13.901279
330,188,2023,0.008354,Volvo,C40,46.6,43.06649,40.2,28,SUV,C,SE,517,1.0,0,0.027496,C40_2023,41.254012
331,189,2023,0.000065,Volvo,EX30,47.5,36.82450,26.8,28,SUV,B,SE,4,1.0,0,0.019355,EX30_2023,34.249576
332,190,2023,0.031752,Volvo,XC40,45.7,43.92666,40.2,28,SUV,C,SE,1965,1.0,0,0.025408,XC40_2023,41.965209


In [70]:
logit_data[logit_data['Cost']>logit_data['Price']]

,ID,Year,Market_share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,Segment,Country,Sales,Intercept,China,CCP,Model_year,Cost
